In [2]:
!pip install --upgrade pip

  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1


In [4]:
!pip install -q PyAthena

In [5]:
import boto3
import sagemaker

original_boto3_version = boto3.__version__
%pip install 'boto3>1.17.21'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
Note: you may need to restart the kernel to use updated packages.


In [6]:
from sagemaker.session import Session

region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [7]:
# You can modify the following to use a bucket of your choosing
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = "sagemaker-featurestore-smartgrid"

print(default_s3_bucket_name)

sagemaker-us-east-1-911199926915


In [8]:
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print(role)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
arn:aws:iam::911199926915:role/LabRole


In [9]:
from pyathena import connect

In [10]:
import pandas as pd
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [11]:
# Set S3 path to Parquet data
s3_path_parquet = "s3://{}/Smart_Grid/data/parquet".format(bucket)

# Set Athena parameters
database_name = "dsoawsfp"
table_name_csv = "meterdata_csv"
table_name_parquet = "meterdata_parquet"

In [12]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [13]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [14]:
year = int(2012)
month = 5

statement = """SELECT * FROM {}.{}
    ORDER BY timestamp""".format(
    database_name, table_name_parquet
)

print(statement)

SELECT * FROM dsoawsfp.meterdata_parquet
    ORDER BY timestamp


In [15]:
smartgrid_feature_group_df = pd.read_sql(statement, conn)
smartgrid_feature_group_df

/tmp/ipykernel_20/3312518796.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  smartgrid_feature_group_df = pd.read_sql(statement, conn)


,timestamp,kwh,weekday,hour,temperature,holiday,year,month
0,2011-11-23 09:00:00,0.284500,2,9.0,4.84,-1.0,2011,11
1,2011-11-23 10:00:00,0.153333,2,10.0,5.74,-1.0,2011,11
2,2011-11-23 11:00:00,0.110286,2,11.0,7.67,-1.0,2011,11
3,2011-11-23 12:00:00,0.394222,2,12.0,8.84,-1.0,2011,11
4,2011-11-23 13:00:00,0.258333,2,13.0,9.87,-1.0,2011,11
...,...,...,...,...,...,...,...,...
19857,2014-02-27 20:00:00,0.347004,3,20.0,5.94,-1.0,2014,2
19858,2014-02-27 21:00:00,0.334706,3,21.0,5.03,-1.0,2014,2
19859,2014-02-27 22:00:00,0.299760,3,22.0,4.10,-1.0,2014,2
19860,2014-02-27 23:00:00,0.252274,3,23.0,3.93,-1.0,2014,2


In [16]:
smartgrid_feature_group_df['timestamp'] = smartgrid_feature_group_df['timestamp'].astype(str)
smartgrid_feature_group_df

,timestamp,kwh,weekday,hour,temperature,holiday,year,month
0,2011-11-23 09:00:00,0.284500,2,9.0,4.84,-1.0,2011,11
1,2011-11-23 10:00:00,0.153333,2,10.0,5.74,-1.0,2011,11
2,2011-11-23 11:00:00,0.110286,2,11.0,7.67,-1.0,2011,11
3,2011-11-23 12:00:00,0.394222,2,12.0,8.84,-1.0,2011,11
4,2011-11-23 13:00:00,0.258333,2,13.0,9.87,-1.0,2011,11
...,...,...,...,...,...,...,...,...
19857,2014-02-27 20:00:00,0.347004,3,20.0,5.94,-1.0,2014,2
19858,2014-02-27 21:00:00,0.334706,3,21.0,5.03,-1.0,2014,2
19859,2014-02-27 22:00:00,0.299760,3,22.0,4.10,-1.0,2014,2
19860,2014-02-27 23:00:00,0.252274,3,23.0,3.93,-1.0,2014,2


In [17]:
from time import gmtime, strftime, sleep

smartgrid_feature_group_name = "smartgrid-feature-group-" + strftime("%d-%H-%M-%S", gmtime())

In [18]:
from sagemaker.feature_store.feature_group import FeatureGroup

smartgrid_feature_group = FeatureGroup(
    name=smartgrid_feature_group_name, sagemaker_session=feature_store_session
)

In [21]:
import time

current_time_sec = int(round(time.time()))


def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == "object":
            data_frame[label] = data_frame[label].astype("str").astype("string")


# cast object dtype to string. The SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.
cast_object_to_string(smartgrid_feature_group_df)

# record identifier and event time feature names
record_identifier_feature_name = "timestamp"
event_time_feature_name = "EventTime"

# append EventTime feature
smartgrid_feature_group_df[event_time_feature_name] = pd.Series(
    [current_time_sec] * len(smartgrid_feature_group_df), dtype="float64"
)

# load feature definitions to the feature group. SageMaker FeatureStore Python SDK will auto-detect the data schema based on input data.
smartgrid_feature_group.load_feature_definitions(data_frame=smartgrid_feature_group_df)

[FeatureDefinition(feature_name='timestamp', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='kwh', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='weekday', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='hour', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='temperature', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='holiday', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='year', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='month', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='EventTime

In [22]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")


smartgrid_feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True,
)

wait_for_feature_group_creation_complete(feature_group=smartgrid_feature_group)

Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup smartgrid-feature-group-05-03-37-06 successfully created.


In [23]:
smartgrid_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:911199926915:feature-group/smartgrid-feature-group-05-03-37-06',
 'FeatureGroupName': 'smartgrid-feature-group-05-03-37-06',
 'RecordIdentifierFeatureName': 'timestamp',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'timestamp', 'FeatureType': 'String'},
  {'FeatureName': 'kwh', 'FeatureType': 'Fractional'},
  {'FeatureName': 'weekday', 'FeatureType': 'Integral'},
  {'FeatureName': 'hour', 'FeatureType': 'Fractional'},
  {'FeatureName': 'temperature', 'FeatureType': 'Fractional'},
  {'FeatureName': 'holiday', 'FeatureType': 'Fractional'},
  {'FeatureName': 'year', 'FeatureType': 'Integral'},
  {'FeatureName': 'month', 'FeatureType': 'Integral'},
  {'FeatureName': 'EventTime', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2024, 2, 5, 3, 42, 11, 412000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://sagemake

In [24]:
sagemaker_client.list_feature_groups() 

{'FeatureGroupSummaries': [{'FeatureGroupName': 'smartgrid-feature-group-05-03-37-06',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:911199926915:feature-group/smartgrid-feature-group-05-03-37-06',
   'CreationTime': datetime.datetime(2024, 2, 5, 3, 42, 11, 412000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'}],
 'ResponseMetadata': {'RequestId': '874bf836-87af-4b24-8505-661443ba1087',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '874bf836-87af-4b24-8505-661443ba1087',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '260',
   'date': 'Mon, 05 Feb 2024 03:43:45 GMT'},
  'RetryAttempts': 0}}

In [25]:
smartgrid_feature_group.ingest(data_frame=smartgrid_feature_group_df, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='smartgrid-feature-group-05-03-37-06', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f18bdc37670>, sagemaker_session=<sagemaker.session.Session object at 0x7f18be6b2980>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7f18ba13eb30>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

In [26]:
record_identifier_value = "2011-11-23 09:00:00"

featurestore_runtime.get_record(
    FeatureGroupName=smartgrid_feature_group_name,
    RecordIdentifierValueAsString=record_identifier_value,
)

{'ResponseMetadata': {'RequestId': '238fe2e5-d261-4c38-8dd3-ef54080af0a4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '238fe2e5-d261-4c38-8dd3-ef54080af0a4',
   'content-type': 'application/json',
   'content-length': '710',
   'date': 'Mon, 05 Feb 2024 03:47:04 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'timestamp',
   'ValueAsString': '2011-11-23 09:00:00'},
  {'FeatureName': 'kwh', 'ValueAsString': '0.2845'},
  {'FeatureName': 'weekday', 'ValueAsString': '2'},
  {'FeatureName': 'hour', 'ValueAsString': '9.0'},
  {'FeatureName': 'temperature', 'ValueAsString': '4.84'},
  {'FeatureName': 'holiday', 'ValueAsString': '-1.0'},
  {'FeatureName': 'year', 'ValueAsString': '2011'},
  {'FeatureName': 'month', 'ValueAsString': '11'},
  {'FeatureName': 'EventTime', 'ValueAsString': '1707104530.0'}]}

In [27]:
print(smartgrid_feature_group.as_hive_ddl())

CREATE EXTERNAL TABLE IF NOT EXISTS sagemaker_featurestore.smartgrid-feature-group-05-03-37-06 (
  timestamp STRING
  kwh FLOAT
  weekday INT
  hour FLOAT
  temperature FLOAT
  holiday FLOAT
  year INT
  month INT
  EventTime FLOAT
  write_time TIMESTAMP
  event_time TIMESTAMP
  is_deleted BOOLEAN
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
  STORED AS
  INPUTFORMAT 'parquet.hive.DeprecatedParquetInputFormat'
  OUTPUTFORMAT 'parquet.hive.DeprecatedParquetOutputFormat'
LOCATION 's3://sagemaker-us-east-1-911199926915/sagemaker-featurestore-smartgrid/911199926915/sagemaker/us-east-1/offline-store/smartgrid-feature-group-05-03-37-06-1707104531/data'


In [28]:
s3_client = boto3.client("s3", region_name=region)

In [30]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
print(account_id)

smartgrid_feature_group_resolved_output_s3_uri = (
    smartgrid_feature_group.describe()
    .get("OfflineStoreConfig")
    .get("S3StorageConfig")
    .get("ResolvedOutputS3Uri")
)

smartgrid_feature_group_s3_prefix = smartgrid_feature_group_resolved_output_s3_uri.replace(
    f"s3://{default_s3_bucket_name}/", ""
)

offline_store_contents = None
while offline_store_contents is None:
    objects_in_bucket = s3_client.list_objects(
        Bucket=default_s3_bucket_name, Prefix=smartgrid_feature_group_s3_prefix
    )
    if "Contents" in objects_in_bucket and len(objects_in_bucket["Contents"]) > 1:
        offline_store_contents = objects_in_bucket["Contents"]
    else:
        print("Waiting for data in offline store...\n")
        sleep(60)

print("Data available.")

911199926915
Waiting for data in offline store...

Data available.


In [31]:
smartgrid_query = smartgrid_feature_group.athena_query()

smartgrid_table = smartgrid_query.table_name

In [33]:
query_string = f'''
SELECT *
FROM 
    "{smartgrid_table}"
'''

print("Running " + query_string)

Running 
SELECT *
FROM 
    "smartgrid_feature_group_05_03_37_06_1707104531"



In [ ]:
# run Athena query. The output is loaded to a Pandas dataframe.
# dataset = pd.DataFrame()
smartgrid_query.run(
    query_string=query_string,
    output_location="s3://" + default_s3_bucket_name + "/" + prefix + "/query_results/",
)
smartgrid_query.wait()
dataset = smartgrid_query.as_dataframe()

dataset

In [ ]:
smartgrid_feature_group.delete()

In [17]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>